<a href="https://colab.research.google.com/github/SIDED00R/machinelearning/blob/main/deep4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')
xy=np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/diabetes.csv.gz', delimiter=',', dtype=np.float32) 
x_data=from_numpy(xy[:, 0:-1])
y_data=from_numpy(xy[:, [-1]])
print(f'X\'s shape: {x_data.shape} | Y\'s shape: {y_data.shape}')

Mounted at /content/gdrive
X's shape: torch.Size([759, 8]) | Y's shape: torch.Size([759, 1])


In [2]:
from torch import nn, optim, from_numpy
import numpy as np
from google.colab import drive
from torch.utils.data import Dataset, DataLoader
drive.mount('/content/gdrive')
class DiabetesDataset(Dataset):
  def __init__(self):
    xy=np.loadtxt('/content/gdrive/MyDrive/Colab Notebooks/diabetes.csv.gz', delimiter=',', dtype=np.float32) 
    self.x_data=from_numpy(xy[:, 0:-1])
    self.y_data=from_numpy(xy[:, [-1]])
    self.len=xy.shape[0]

    

  def __getitem__(self,index):
      return self.x_data[index], self.y_data[index]

  def __len__(self):
      return self.len

dataset=DiabetesDataset()
train_loader=DataLoader(dataset= dataset, batch_size=32, shuffle=True, num_workers=2)

class Model(nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.l1=nn.Linear(8,6)
    self.l2=nn.Linear(6,4)
    self.l3=nn.Linear(4,1)
    self.sigmoid=nn.Sigmoid()
    self.relu=nn.ReLU()

  def forward(self,x):
    out1=self.relu(self.l1(x))
    out2=self.relu(self.l2(out1))
    y_pred=self.sigmoid(self.l3(out2))
    return y_pred

model=Model()
criterion=nn.BCELoss(reduction='mean')
optimizer=optim.SGD(model.parameters(), lr=0.1)
for epoch in range(100):
  for i, data in enumerate(train_loader,0):
    inputs,labels=data
    y_pred=model(inputs)
    loss=criterion(y_pred,labels)
    print(f'Epoch{epoch+1} | Batch: {i+1} Loss: {loss.item(): .4f}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Epoch1 | Batch: 1 Loss:  0.7114
Epoch1 | Batch: 2 Loss:  0.7166
Epoch1 | Batch: 3 Loss:  0.7251
Epoch1 | Batch: 4 Loss:  0.6984
Epoch1 | Batch: 5 Loss:  0.7136
Epoch1 | Batch: 6 Loss:  0.6986
Epoch1 | Batch: 7 Loss:  0.6995
Epoch1 | Batch: 8 Loss:  0.6937
Epoch1 | Batch: 9 Loss:  0.6958
Epoch1 | Batch: 10 Loss:  0.6963
Epoch1 | Batch: 11 Loss:  0.6923
Epoch1 | Batch: 12 Loss:  0.6891
Epoch1 | Batch: 13 Loss:  0.6824
Epoch1 | Batch: 14 Loss:  0.6886
Epoch1 | Batch: 15 Loss:  0.6819
Epoch1 | Batch: 16 Loss:  0.6701
Epoch1 | Batch: 17 Loss:  0.6805
Epoch1 | Batch: 18 Loss:  0.6837
Epoch1 | Batch: 19 Loss:  0.6707
Epoch1 | Batch: 20 Loss:  0.6735
Epoch1 | Batch: 21 Loss:  0.6711
Epoch1 | Batch: 22 Loss:  0.6701
Epoch1 | Batch: 23 Loss:  0.6629
Epoch1 | Batch: 24 Loss:  0.6678
Epoch2 | Batch: 1 Loss:  0.6466
Epoch2 | Batch: 2 Loss:  0.6924
Epoch2 | Batch: 3 Loss